In [11]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mtree import MTree
from plotting import set_plot_datapath, plot_minutia_zoom, plot_minutia_compare, imshow_sample, mark_minutia
import csv
import os

mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{{amsmath}}'
datapath = '../data/fvc2002/'
set_plot_datapath(datapath)

In [12]:
def get_data() -> list:
    gallery = []
    for i in range(1,11):
        for j in range(1,9):
            if (i, j) == (10, 4):
                # Dado problemático
                continue
            
            filename = datapath + f'features/A{i}_{j}.npy'
            features = np.load(filename)

            for k in range(features.shape[0]):
                gallery.append({'finger': i, 'sample': j, 'minutia_no': k, 'vec': features[k, :]})
    return gallery

def Dcos(x, y):
    eps = 1e-5 # Ensure positivity
    return 1 + eps - np.dot(x['vec'], y['vec'])

def L2(x, y):
    return np.linalg.norm(x['vec'] - y['vec'])

def matchDict2csv(matchDict: dict, filename: str) -> None:
    matchDict = dict(sorted(matchDict.items(), key=lambda item: item[1], reverse=True))

    # Abrindo o arquivo CSV para escrita
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Escrevendo o cabeçalho
        writer.writerow(['Sample Name', 'Value'])
        
        # Escrevendo os dados
        for sample_name, value in matchDict.items():
            writer.writerow([sample_name, value])

def gallery_search(finger, sample):
    
    # Para cada minúcia, encontrar os k mais próximos
    filename = datapath + f'features/A{finger}_{sample}.npy'
    features = np.load(filename)

    sample_features = []
    for k in range(features.shape[0]):
        sample_features.append({'finger': finger, 'sample': sample, 'minutia_no': k, 'vec': features[k, :]})
    
    results = []
    for feature in sample_features:
        k = 8
        results.append(list(tree.search(feature, k)))
        
    # Agrupar os resultados de cada minúcia
    matchDict = dict()
    for minutia_no in range(len(sample_features)):
        for r in results[minutia_no]:
            if r is None:
                continue
            key = f"A{r['finger']}_{r['sample']}"
            matchDict[key] = matchDict.get(key, 0) + 1 # Score

    matchDict2csv(matchDict, f'results/cosineDist/A{finger}_{sample}.csv')

In [13]:
gallery = get_data()

tree = MTree(Dcos, max_node_size=8)
tree.add_all(gallery)

for f in range(1,11):
    for s in range(1,9):
        if (f, s) == (10, 4):
            # Dado problemático
            continue
        gallery_search(f, s)

In [16]:
def process_file(file_path):
    global counter
    if file_path.endswith('.csv'):
        with open(file_path, mode='r') as file:
            csv_reader = csv.reader(file)
            header = next(csv_reader)  # Skip the header
            
            # Read the first two lines
            first_line = next(csv_reader, None)
            second_line = next(csv_reader, None)
            if first_line and second_line:
                id1 = first_line[0].split('_')[0][1:]
                id2 = second_line[0].split('_')[0][1:]
                if id1 == id2:
                    counter += 1
                
            ids = []
            # for i in range(7):
            #     line = next(csv_reader, None)
            #     if line:
            #         id = line[0].split('_')[0][1:]
            #         ids.append(id)
            #     else:
            #         break
                
            return tuple(ids)

def process_dir(directory):
    results = []
    
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        process_file(file_path)
        #results.append(res)
        
    return results

In [18]:
directory_path = 'results/cosineDist'
counter = 0
results = process_dir(directory_path)
counter

79